# Lectura 48: Selectores

## Selectores

Los selectores permiten una selección más intuitiva de las columnas de los DataFrame o LazyFrame en función de su nombre, tipo de dato u otras propiedades.

In [1]:
import polars as pl

salarios = pl.DataFrame(
    {
        'lenguaje': ['Python', 'Scala', 'C', 'Python', 'Scala', 'Java'],
        'puesto': ['Sr', 'Sr', 'Jr', 'Jr', 'Sr', 'Jr'],
        'salario': [5000.00, 7000.00, 10000.00, 4500.00, 5400.00, 6000.00],
        'experiencia': [3, 5, 8, 2, 4, 2]
    }
)

### ¿Cómo importarlos?

- Los selectores están disponibles como funciones importadas desde `polars.selectors`
- El uso típico/recomendado es importar el módulo como `cs` y emplear los selectores desde allí

In [2]:
import polars.selectors as cs

salarios.group_by(cs.string()).agg(
    cs.numeric().mean().name.suffix('_avg')
)

lenguaje,puesto,salario_avg,experiencia_avg
str,str,f64,f64
"""Scala""","""Sr""",6200.0,4.5
"""Python""","""Jr""",4500.0,2.0
"""Java""","""Jr""",6000.0,2.0
"""Python""","""Sr""",5000.0,3.0
"""C""","""Jr""",10000.0,8.0


## Operaciones

In [3]:
operaciones = pl.DataFrame(
    schema={
        "a": pl.UInt32,
        "b": pl.Float64,
        "c": pl.Float32,
        "d": pl.Boolean,
        "e": pl.Time,
        "f": pl.Date,
        "g": pl.Duration,
        "h": pl.Datetime("ms"),
        "i": pl.String,
    },
)

### Unión

In [4]:
operaciones.select(
    cs.numeric() | cs.string()
)

a,b,c,i
u32,f64,f32,str


### Intersección

In [5]:
operaciones.select(
    cs.temporal() & cs.matches('g|h')
)

g,h
duration[μs],datetime[ms]


### Diferencia

In [6]:
operaciones.select(
    cs.numeric() - cs.first()
)

b,c
f64,f32


### Negación

In [7]:
operaciones.select(
    ~cs.numeric()
)

d,e,f,g,h,i
bool,time,date,duration[μs],datetime[ms],str
